In [7]:
import os
from icecream import ic
import pandas as pd
from dotenv import load_dotenv
import sqlalchemy
from sqlalchemy import create_engine
load_dotenv()

ic(os.getcwd())

ic| os.getcwd(): '/home/victor/Documents/projects/regreen_cleaning/brics3_migration'


'/home/victor/Documents/projects/regreen_cleaning/brics3_migration'

# The data
two datasets
- shared by unit
- cleaned by providers

**problem**
- cleaned data is missing username column.
- generate json to add halfoom data to regreen db
- 

**solution**
- use plot details as unique key
- check if all keys are unique

### cleaned df

In [14]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_combined_rangeland_somalia_brics3_raf_terra.csv')
cleaned_df = pd.read_csv(cleaned_csv)
#set(cleaned_df.columns)

In [17]:
cleaned_df['geometry'].nunique()

120

In [18]:
list(cleaned_df[cleaned_df['geometry'].duplicated()]['geometry'].unique())

[nan]

### shared df

In [19]:
shared_csv = os.path.join('data', 'rangeland_somalia_brics3_area_final_data_shared.csv')
shared_df = pd.read_csv(shared_csv)
#set(shared_df.columns)
#shared_df['geometry'].nunique()
shared_df['geometry'].nunique()

179

In [20]:
y = set(shared_df.columns) - set(cleaned_df.columns)
y

{'AttachmentsExpected',
 'AttachmentsPresent',
 'DeviceID',
 'Edits',
 'FormVersion',
 'KEY',
 'ReviewState',
 'Status',
 'SubmissionDate',
 'SubmitterID',
 'SubmitterName',
 'basic_info-name',
 'end',
 'meta-instanceID',
 'start'}

### join the dfs based on geometry(plot) column
left merge: 
- it keeps all the data from the left DataFrame and adds matching information from the right DataFrame
- https://medium.com/@whyamit404/understanding-left-merge-in-pandas-03f04cd3cf5f

In [21]:
merged_df = pd.merge(cleaned_df, shared_df[['geometry','basic_info-name']], on='geometry', how='left')
#merged_df.to_csv(os.path.join(data_folder, 'output2.csv'))
merged_df

,basic_info-date,basic_info-organisation_name,basic_info-organisation_other,geography-state_name,geography-region_name,geography-district_name,geography-community_name,plot_details-photo,plot_details-polygon,land_use_selection-project_name,...,waterpoints-wp_facilities,waterpoints-gps-Latitude,waterpoints-gps-Longitude,waterpoints-gps-Altitude,waterpoints-gps-Accuracy,waterpoints-photo,geometry,area_m2,area_ha,basic_info-name
0,15/05/2025,concern,NaN,jubaland.,gedo,baardhere,buulo dhato,1749537712661.jpg,2.352898 42.2954533 70.3623046875 7.324;2.3536...,brcis3,...,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((42.29545 2.352898, 42.29368 2.353665...",43376.958180,4.337696,Abdullahi
1,29/05/2025,irc,NaN,galmudug,mudug,galkacyo,dhardhar,1748511134226.jpg,6.6435158 46.9506466 320.0 2.0;6.6436798 46.95...,brcis3,...,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((46.95065 6.643516, 46.95053 6.64368,...",168417.837100,16.841784,Ahmed Omar Hassan
2,17/05/2025,kaalo,NaN,puntland,mudug.,galkacyo.,gacmo fale,1747480392579.jpg,7.0243358 47.7520651 288.0 7.5; 7.024208 47.75...,brcis3,...,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((47.75207 7.024336, 47.75187 7.024208...",130.744739,0.013074,Abdijalil
3,17/05/2025,kaalo,NaN,puntland,mudug.,galkacyo.,gacmo fale,1747479455200.jpg,7.0367036 47.7713796 285.0 3.0;7.036829 47.771...,brcis3,...,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((47.77138 7.036704, 47.77153 7.036829...",66.898016,0.006690,Abdijalil
4,17/05/2025,kaalo,NaN,puntland,mudug.,galkacyo.,gacmo fale,1747478430265.jpg,7.0382631 47.7670073 281.0 3.682; 7.0383558 47...,brcis3,...,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((47.76701 7.038263, 47.76692 7.038356...",1283.520921,0.128352,Abdijalil said
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,12/10/2024,IRC,NaN,galmudug,Mudug,Galkacyo,Dhardhar,NaN,NaN,brcis3,...,NaN,6.644095,46.951650,323.7,4.725,NaN,NaN,NaN,NaN,NaN
304,12/23/2024,SCI,NaN,hirshabele,Hiran,BeletWeyne,Jawiil,NaN,NaN,brcis3,...,NaN,4.917423,45.221193,170.8,4.400,NaN,NaN,NaN,NaN,NaN
305,12/23/2024,SCI,NaN,hirshabele,Hiran,BeletWeyne,Jawiil,NaN,NaN,brcis3,...,NaN,4.923309,45.223448,176.0,2.400,NaN,NaN,NaN,NaN,NaN
306,12/23/2024,SCI,NaN,hirshabele,Hiran,BeletWeyne,Warqumayo,NaN,NaN,brcis3,...,NaN,4.994899,45.160282,199.5,3.600,NaN,NaN,NaN,NaN,NaN


### get names from db

In [22]:
# create a db engine
engine = create_engine(os.getenv('DATABASE_URL1'))

#query 
query = '''select id, username, first_name, last_name, email, country from respi_regreeningusers where country in ('Somalia', 'somalia') order by first_name asc;'''

#execute the query and fetch the results
conn = engine.connect()
df = pd.read_sql_query(query, conn)

#close the connection
conn.close()
engine.dispose()

df

,id,username,first_name,last_name,email,country
0,3733,abdikafim,,,abdikafi.mohamud@cpd-africa.org,Somalia
1,4307,Abdalla.MARDO,Abdalla,Hassan,ganacsadeabdalla@gmail.com,Somalia
2,4546,hilal,Abdalla,Hassan,hilalgs106@gmail.com,Somalia
3,4311,AbdiDubow,Abdi,Dubow,Abdijelledubow77@gmail.com,Somalia
4,1844,adandugsi,Abdi,Aden,abdi.adan@nrc.no,Somalia
...,...,...,...,...,...,...
170,3203,Saiid,Saiid,Abdirahman,Zacksupdi@gmail.com,Somalia
171,1834,Shueb,Shueb,Nur,Shueb.nur@savethechildren.org,Somalia
172,4305,sundus,sundus,Mohamed,sundus.ali@kaalo.org,Somalia
173,3725,Mcnbdn617660,Yusuf,Mahi,yusuf.munye@hijra.or.ke,Somalia


### get unique names from merged df

In [23]:
merged_df.drop_duplicates(subset=['basic_info-name'])[['basic_info-name', 'basic_info-organisation_name','land_use_selection-intervention']]
#merged_df[['basic_info-name', 'basic_info-organisation_name','land_use_selection-intervention']]

,basic_info-name,basic_info-organisation_name,land_use_selection-intervention
0,Abdullahi,concern,soil_bunds
1,Ahmed Omar Hassan,irc,halfmoons
2,Abdijalil,kaalo,gabions
4,Abdijalil said,kaalo,gabions
5,Maria Abdirahman Ali,irc,halfmoons
7,NaN,nrc,halfmoons
8,Ahmed Ibrahim,nrc,swales
12,Hassan Omar Khadhib,nrc,water_points
13,Hassan Omar Khadhibr,nrc,halfmoons water_points
14,ABSHIR ABDULLAH ALI,nrc,halfmoons contour_planting water_points rock_dam


### present names(in db) - halfmoons
- Jamal Ali Tagal - 3228, acf
- Muktar Mohamednoor Rage - 1831, acf
- Abdiaziz Adan Hassan - 3726, gredo
- Abdikarim Mohamed Ismail - 4247, gredo 
- Abdirashid Sheikh Mohamed - 4548, gredo

### present names - water_points
- Hassan Omar Khadhib - 2495, nrc

### missing names(in db) - halfmoons
- Ahmed Omar Hassan - irc, 
- Maria Abdirahman Ali - irc
- NAN - nrc
- ABSHIR ABDULLAH ALI - nrc
- Mohamud Abdullahi Ali - nrc
- Moulid Abdullahi Abdi - nrc
- Zakariye haji ali - acf
- Abdullahi Hassan Adan - acf
- Abdi Hassan Adan - acf
- Issack Idow Ibrahim - acf
- 

### missing names - soil bunds
- Abdullahi - concern
- Ali - concern
- Fathi - concern
- Fathi Abdirashid - concern

### missing names - gabions
- Abdijalil Said - kaalo

### missing names - swales
- Ahmed Ibrahim - nrc

### missing names - water_points
- Ali Aden - concern
- Abdullahi Sharif LLG - concern
- Abdikheir Mohamed Ali - concern
- Abdikheir Mohamed Dahir - concern
- Ahmed Ibrahim Mohamed - nrc
- Yusuf Mahi munye - concern
- Osman Ebey Omar - concern





## work on wps(no geom)
- get the waterpoints(those that dont have polygon geometry)


In [24]:
wps = cleaned_df[cleaned_df['geometry'].isnull()]
wps.to_csv(os.path.join(data_folder, 'waterpoints_no_polygon.csv'))

## get waterpoints that intersect with plot geometry.
in qgis, >> vector >> data management tools >> join attributes by location
- join the attributes of waterpoints layer(with no plot geometry) to those of the plot-polygons layer.
- now you have the key('plot-details_photo') to left merge.
- why this key?
    - in shared df, it is present and has collectyor names

In [25]:
wps_intersecting = pd.read_csv(os.path.join(data_folder, '.csv'))
wps_intersecting

FileNotFoundError: [Errno 2] No such file or directory: '/home/victor/Documents/projects/regreen_cleaning/brics3_migration/data/.csv'

In [57]:
#list(wps_intersecting.columns)

In [54]:
# wps_merged = pd.merge(wps_intersecting, shared_df[['plot_details-photo','basic_info-name']], on='plot_details-photo', how='left')
# wps_merged.to_csv(os.path.join(data_folder, 'wps_merged.csv'))
# wps_merged.head()
shared_df[['plot_details-photo','basic_info-name']]

,plot_details-photo,basic_info-name
0,1756980579324.jpg,Abdikarim Mohamed Ismail
1,1756980161302.jpg,Abdikarim Mohamed Ismail
2,1756978271125.jpg,Abdikarim Mohamed Ismail
3,1756889048200.jpg,Ahmed Ibrahim
4,1756794863604.jpg,Osman Ebey Omar
...,...,...
174,1736667942332.jpg,Jamal Ali
175,1736444756707.jpg,ICRAF
176,1733305063232.jpg,Muhammad
177,1729969831010.jpg,Jasper


## inspect halfmoons json

In [62]:
y

{'AttachmentsExpected',
 'AttachmentsPresent',
 'DeviceID',
 'Edits',
 'FormVersion',
 'KEY',
 'ReviewState',
 'Status',
 'SubmissionDate',
 'SubmitterID',
 'SubmitterName',
 'basic_info-name',
 'end',
 'meta-instanceID',
 'start'}

In [27]:
#cleaned_df['land_use_selection-project_name'].unique()
list(cleaned_df.columns)
#cleaned_df.head()

['basic_info-date',
 'basic_info-organisation_name',
 'basic_info-organisation_other',
 'geography-state_name',
 'geography-region_name',
 'geography-district_name',
 'geography-community_name',
 'plot_details-photo',
 'plot_details-polygon',
 'land_use_selection-project_name',
 'land_use_selection-land_use',
 'land_use_selection-intervention',
 'halfmoons-halfmoon_details-halfmoon_type',
 'halfmoons-halfmoon_details-date',
 'halfmoons-halfmoon_details-total_halfmoons',
 'halfmoons-halfmoon_details-size',
 'halfmoons-halfmoon_details-length_decimal',
 'halfmoons-halfmoon_details-width_decimal',
 'halfmoons-halfmoon_details-depth_decimal',
 'halfmoons-halfmoon_details_2-type_halfmoons',
 'halfmoons-halfmoon_details_2-average_space',
 'halfmoons-halfmoon_details_2-vertical_space',
 'halfmoons-halfmoon_details_2-horizontal_space',
 'halfmoons-halfmoon_details_2-manages',
 'halfmoons-halfmoon_details_2-youth_manages',
 'halfmoons-halfmoon_details_2-paid_unpaid_labor',
 'swales-swales_detai

In [60]:
# import json
# with open(os.path.join(data_folder,'halfmoon_json_template.json')) as file:
#     data = json.load(file)[0]

# list(data.keys())

list(shared_df.columns)

['SubmissionDate',
 'start',
 'basic_info-name',
 'basic_info-date',
 'basic_info-organisation_name',
 'basic_info-organisation_other',
 'geography-state_name',
 'geography-region_name',
 'geography-district_name',
 'geography-community_name',
 'plot_details-photo',
 'plot_details-polygon',
 'land_use_selection-project_name',
 'land_use_selection-land_use',
 'land_use_selection-intervention',
 'halfmoons-halfmoon_details-halfmoon_type',
 'halfmoons-halfmoon_details-date',
 'halfmoons-halfmoon_details-total_halfmoons',
 'halfmoons-halfmoon_details-size',
 'halfmoons-halfmoon_details-length_decimal',
 'halfmoons-halfmoon_details-width_decimal',
 'halfmoons-halfmoon_details-depth_decimal',
 'halfmoons-halfmoon_details_2-type_halfmoons',
 'halfmoons-halfmoon_details_2-average_space',
 'halfmoons-halfmoon_details_2-vertical_space',
 'halfmoons-halfmoon_details_2-horizontal_space',
 'halfmoons-halfmoon_details_2-manages',
 'halfmoons-halfmoon_details_2-youth_manages',
 'halfmoons-halfmoon_de

# nested dictionaries are
- 